In [1]:
from scipy.sparse import save_npz

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import re
import numpy as np

# ... (загрузка данных и функция normalize_address остаются без изменений) ...
# Начало скрипта prepare_database_ngram.py

df = pd.read_csv('df_cleaned.csv')
# Используем тот же пример данных для консистентности
df['address'] = df['street'] + ' ' + df['housenumber']

def normalize_address(address):
    """
    Приводит адрес к стандартному, "чистому" виду:
    - Нижний регистр, обработка буквы "ё".
    - Удаление знаков препинания.
    - Стандартизация сокращений (ул, г, д и т.д.).
    - Удаление всех пробелов для слитного написания.
    """
    # 1. Приводим к нижнему регистру и обрабатываем "ё"
    address = address.lower().replace('ё', 'е')
    
    # 2. Удаляем все символы, кроме букв, цифр и пробелов
    address = re.sub(r'[^а-яa-z0-9\s/]', ' ', address)
    
    # 3. Стандартизация ключевых сокращений
    replacements = {
        r'\bг\b': 'город', r'\bул\b': 'улица', r'\bд\b': 'дом',
        r'\bпр\b': 'проспект', r'\bпр-т\b': 'проспект', r'\bк\b': 'корпус',
        r'\bкорп\b': 'корпус', r'\bстр\b': 'строение', r'\bлит\b': 'литера'
    }
    for old, new in replacements.items():
        address = re.sub(old, new, address)
        
    # 4. Схлопываем множественные пробелы в один (промежуточный шаг)
    address = ' '.join(address.split())
    
    # # 5. НОВЫЙ ШАГ: Удаляем ВСЕ пробелы, чтобы получить слитное написание
    # address = re.sub(r'\s+', '', address)
    
    return address

df['normalized'] = df['address'].apply(normalize_address)

# --- Создание TF-IDF векторов ---
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3),encoding='cp1251')
tfidf_matrix = vectorizer.fit_transform(df['normalized'])
print(f"Матрица TF-IDF создана. Размер: {tfidf_matrix.shape}")
print(f"( {tfidf_matrix.shape[0]} адресов, {tfidf_matrix.shape[1]} уникальных 3-грамм )")

# --- Шаг 3: Сохранение артефактов ---

# Сохраняем "обученный" векторизатор
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Сохраняем TF-IDF матрицу (векторы)
# Используем специальный формат для разреженных матриц
save_npz('tfidf_matrix.npz', tfidf_matrix)

# Сохраняем сами данные для сопоставления
df.to_pickle('address_data_ngram.pkl')

Матрица TF-IDF создана. Размер: (253680, 9837)
( 253680 адресов, 9837 уникальных 3-грамм )


In [3]:
feature_names = vectorizer.get_feature_names_out()

# Получаем массив весов IDF
idf_scores = vectorizer.idf_

# Создаем DataFrame для удобного анализа
idf_df = pd.DataFrame({
    'ngram': feature_names,
    'idf_score': idf_scores
})

# --- ШАГ 4: Вывод результатов ---

# Сортируем по весу IDF по убыванию, чтобы увидеть самые ВАЖНЫЕ
idf_df_sorted_desc = idf_df.sort_values(by='idf_score', ascending=False)

print("\n\n===== ТОП-20 САМЫХ ВАЖНЫХ (УНИКАЛЬНЫХ) N-ГРАММ =====")
print(idf_df_sorted_desc.head(20).to_string(index=False))


# # Сортируем по весу IDF по возрастанию, чтобы увидеть самые НЕВАЖНЫБЕ (ЧАСТЫЕ)
# idf_df_sorted_asc = idf_df.sort_values(by='idf_score', ascending=True)

# print("\n\n===== ТОП-20 САМЫХ НЕВАЖНЫХ (ЧАСТЫХ) N-ГРАММ =====")
# print(idf_df_sorted_asc.head(20).to_string(index=False))



===== ТОП-20 САМЫХ ВАЖНЫХ (УНИКАЛЬНЫХ) N-ГРАММ =====
ngram  idf_score
   00  12.750686
  яул  12.750686
  яче  12.750686
  аяу  12.750686
  б 4  12.750686
  люш  12.750686
  31ю  12.750686
  31п  12.750686
  31н  12.750686
  31и  12.750686
  31ж  12.750686
  32е  12.750686
  31a  12.750686
   лф  12.750686
  335  12.750686
  32т  12.750686
  в10  12.750686
  в/3  12.750686
  ас6  12.750686
  в о  12.750686


In [85]:
idf_df_sorted_asc

,ngram,idf_score
0,,1.000000
4949,и,1.081026
3295,а,1.085855
5727,л,1.091191
8159,у,1.134716
...,...,...
7576,с3в,12.750686
1986,44д,12.750686
1968,43ж,12.750686
1967,43д,12.750686


In [86]:
df

,id,latitude,longitude,housenumber,street,address,normalized
0,389567667,55.918906,37.818866,27/21,улица карла маркса,улица карла маркса 27/21,улица карла маркса 27/21
1,389567701,55.920442,37.819326,23/16,улица карла маркса,улица карла маркса 23/16,улица карла маркса 23/16
2,389582528,55.920523,37.818495,16,улица карла маркса,улица карла маркса 16,улица карла маркса 16
3,623415761,55.741123,37.710229,12А с11,подъёмная улица,подъёмная улица 12А с11,подъемная улица 12а с11
4,729316625,55.874163,37.636353,9А,полярная улица,полярная улица 9А,полярная улица 9а
...,...,...,...,...,...,...,...
253675,19824311,55.751038,37.450313,7,улица нижние мнёвники,улица нижние мнёвники 7,улица нижние мневники 7
253676,19824844,55.749064,37.458265,20,улица нижние мнёвники,улица нижние мнёвники 20,улица нижние мневники 20
253677,19824855,55.743981,37.461061,12А,улица нижние мнёвники,улица нижние мнёвники 12А,улица нижние мневники 12а
253678,19824856,55.761190,37.471396,70,улица нижние мнёвники,улица нижние мнёвники 70,улица нижние мневники 70


In [4]:
import pandas as pd
import numpy as np
import pickle
import re

# Подавление будущих предупреждений от scikit-learn
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    from scipy.sparse import load_npz
except ImportError:
    print("Ошибка: Библиотека scikit-learn не установлена.")
    print("Выполните: pip install scikit-learn")
    exit()

# --- Шаг 1: Загрузка артефактов N-грамм ---

try:
    print("Загрузка модели N-грамм (TF-IDF)...")
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
    tfidf_matrix = load_npz('tfidf_matrix.npz')
    df = pd.read_pickle('address_data_ngram.pkl')
    print("✅ Модель успешно загружена.")
except FileNotFoundError:
    print("\n❌ Ошибка: Не найдены файлы для модели N-грамм!")
    print("Пожалуйста, сначала запустите скрипт 'prepare_database_ngram.py' для их создания.")
    exit()
    
def find_coordinates_ngram(query, score_threshold=0.5):
    """
    Выполняет поиск адреса с помощью метода N-грамм / TF-IDF.
    """
    # 1. Нормализация и векторизация запроса
    normalized_query = normalize_address(query)
    query_vector = vectorizer.transform([normalized_query])

    # 2. Вычисление косинусного сходства со всей базой
    similarities = cosine_similarity(query_vector, tfidf_matrix)
    scores = similarities[0]

    # 3. Нахождение лучшего совпадения
    best_match_index = np.argmax(scores)
    highest_score = scores[best_match_index]

    # 4. Проверка порога и возврат результата
    if highest_score >= score_threshold:
        result = df.loc[best_match_index]
        return {
            'found_address': result['full_address'],
            'latitude': result['latitude'],
            'longitude': result['longitude'],
            'score': f"{highest_score:.4f} (cosine similarity)"
        }
    else:
        return None

# --- Шаг 3: Основной цикл для ручной проверки ---


Загрузка модели N-грамм (TF-IDF)...
✅ Модель успешно загружена.


In [88]:
!pip install rapidfuzz

In [89]:
!pip install faiss-cpu

In [5]:

import pandas as pd
import pickle
import re
from scipy.sparse import load_npz
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from rapidfuzz import fuzz # Импортируем rapidfuzz

# --- Блок загрузки (остается без изменений) ---
print("Загрузка сервиса...")
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
tfidf_matrix = load_npz('tfidf_matrix.npz')
df = pd.read_pickle('address_data_ngram.pkl')
print("Сервис готов к работе.")

Загрузка сервиса...
Сервис готов к работе.


In [92]:
df

,id,latitude,longitude,housenumber,street,address,normalized
0,389567667,55.918906,37.818866,27/21,улица карла маркса,улица карла маркса 27/21,улица карла маркса 27/21
1,389567701,55.920442,37.819326,23/16,улица карла маркса,улица карла маркса 23/16,улица карла маркса 23/16
2,389582528,55.920523,37.818495,16,улица карла маркса,улица карла маркса 16,улица карла маркса 16
3,623415761,55.741123,37.710229,12А с11,подъёмная улица,подъёмная улица 12А с11,подъемная улица 12а с11
4,729316625,55.874163,37.636353,9А,полярная улица,полярная улица 9А,полярная улица 9а
...,...,...,...,...,...,...,...
253675,19824311,55.751038,37.450313,7,улица нижние мнёвники,улица нижние мнёвники 7,улица нижние мневники 7
253676,19824844,55.749064,37.458265,20,улица нижние мнёвники,улица нижние мнёвники 20,улица нижние мневники 20
253677,19824855,55.743981,37.461061,12А,улица нижние мнёвники,улица нижние мнёвники 12А,улица нижние мневники 12а
253678,19824856,55.761190,37.471396,70,улица нижние мнёвники,улица нижние мнёвники 70,улица нижние мневники 70


In [6]:
def find_coordinates_sparse_hybrid(query, k=10, score_threshold=85): # ИЗМЕНЕНИЕ 1: более адекватный порог
    """
    Выполняет гибридный поиск:
    1. Находит K кандидатов по TF-IDF (быстро, на разреженной матрице).
    2. Ранжирует их с помощью fuzz.token_sort_ratio (точно, нечувствительно к порядку слов).
    """
    # --- ЭТАП 1: Быстрый поиск кандидатов ---
    normalized_query = normalize_address(query)
    query_vector = vectorizer.transform([normalized_query])

    scores = (tfidf_matrix * query_vector.T).toarray()
    
    # ИЗМЕНЕНИЕ 2: "Расплющиваем" scores в 1D массив перед сортировкой
    scores_1d = scores.flatten()
    
    # Сортируем и берем K лучших индексов
    candidate_indices = np.argsort(scores_1d)[-k:]
    candidate_indices = np.flip(candidate_indices) # Разворачиваем, чтобы лучший был первым

    # --- ЭТАП 2: Точное ранжирование кандидатов ---
    best_match_idx = -1 # Инициализируем как -1 на случай, если ничего не найдется
    highest_score = 0

    for idx in candidate_indices:
        # Пропускаем кандидатов с нулевой схожестью на первом этапе
        if scores_1d[idx] == 0:
            continue
            
        candidate_address = df.loc[idx, 'normalized']
        
        # Вычисляем точную схожесть, нечувствительную к порядку слов
        score = fuzz.token_sort_ratio(normalized_query, candidate_address)
        
        if score > highest_score:
            highest_score = score
            best_match_idx = idx

    # --- ЭТАП 3: Проверка порога и возврат результата ---
    if highest_score >= score_threshold:
        result_row = df.loc[best_match_idx]
        return {
            # ИЗМЕНЕНИЕ 3: Используйте правильные имена колонок из вашего DataFrame!
            'found_address': result_row['address'], 
            'latitude': result_row['latitude'],
            'longitude': result_row['longitude'],
            # ИЗМЕНЕНИЕ 4: Правильная метка для score
            'score': f"{highest_score} (fuzz.token_sort_ratio)" 
        }
    else:
        return None
# --- Тестирование ---
result = find_coordinates_sparse_hybrid("Чистяковой улица дом 1")
print(result)

{'found_address': 'улица чистяковой 1', 'latitude': np.float64(55.69532), 'longitude': np.float64(37.33329), 'score': '90.0 (fuzz.token_sort_ratio)'}


In [12]:
while True:
    user_query = input("\n> Введите адрес: ")
    if user_query.lower() in ['выход', 'exit']:
        break

    # Ищем результат
    result = find_coordinates_sparse_hybrid(user_query, score_threshold=0.3)

    print("-------------------------------------------------")
    if result:
        print("✅ Найдено совпадение:")
        for key, value in result.items():
            print(f"  - {key}: {value}")
    else:
        print("❌ Совпадение не найдено (или схожесть ниже порога).")
    print("-------------------------------------------------")


-------------------------------------------------
✅ Найдено совпадение:
  - found_address: ленинская улица 1
  - latitude: 55.150299
  - longitude: 37.417465
  - score: 61.53846153846154 (fuzz.token_sort_ratio)
-------------------------------------------------
-------------------------------------------------
✅ Найдено совпадение:
  - found_address: ленинский проспект 9
  - latitude: 55.724127
  - longitude: 37.605511
  - score: 100.0 (fuzz.token_sort_ratio)
-------------------------------------------------
-------------------------------------------------
✅ Найдено совпадение:
  - found_address: пятницкое шоссе 1
  - latitude: 55.835491
  - longitude: 37.387139
  - score: 83.87096774193549 (fuzz.token_sort_ratio)
-------------------------------------------------
-------------------------------------------------
❌ Совпадение не найдено (или схожесть ниже порога).
-------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [94]:
%%timeit
result = find_coordinates_sparse_hybrid('проспкек мира 67', score_threshold=0.6)
result

72.1 ms ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
!pip install line_profiler

In [ ]:
%load_ext line_profiler
%lprun -f find_coordinates_sparse find_coordinates_sparse("проспкек мира 67", score_threshold=0.6)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Timer unit: 1e-07 s

Total time: 0.0776014 s

Could not find file C:\Users\Егор\AppData\Local\Temp\ipykernel_4176\2835369957.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    34                                           
    35                                           
    36                                           
    37                                           
    38                                           
    39         1        813.0    813.0      0.1  
    40         1      22788.0  22788.0      2.9  
    41                                           
    42                                           
    43                                           
    44                                           
    45         1     746737.0 746737.0     96.2  
    46                                           
    47          

In [11]:
import json

input_path = 'input.txt'
output_path = 'output.txt'

# Читаем входной JSON
try:
    with open(input_path, 'r', encoding='utf-8') as f:
        addresses_to_search = json.load(f)
    if not isinstance(addresses_to_search, list):
        raise ValueError("Входной JSON должен быть списком (массивом) строк.")
except FileNotFoundError:
    print(f"Ошибка: Входной файл не найден по пути: {input_path}")
except (json.JSONDecodeError, ValueError) as e:
    print(f"Ошибка: Не удалось прочитать или распарсить входной JSON файл. {e}")

print(f"\nНачинаю обработку {len(addresses_to_search)} адресов из файла {input_path}...")

final_results = []

# Обрабатываем каждый адрес из списка
for i, address_query in enumerate(addresses_to_search):
    found_objects = find_coordinates_sparse_hybrid(address_query)
    
    result_for_query = {
        "searched_address": address_query,
        "objects": found_objects
    }
    final_results.append(result_for_query)

# Записываем результат в выходной JSON
try:
    with open(output_path, 'w', encoding='utf-8') as f:
        # ensure_ascii=False для корректного отображения кириллицы
        # indent=2 для красивого форматирования
        json.dump(final_results, f, ensure_ascii=False, indent=2)
except IOError:
    print(f"Ошибка: Не удалось записать результат в файл: {output_path}")
print(f"Результаты сохранены в файл: {output_path}")


Начинаю обработку 4 адресов из файла input.txt...
Результаты сохранены в файл: output.txt


In [1]:
from openai import OpenAI

In [27]:
api_key = 'http://127.0.0.1:11434/v1'
model_name = 'gemma3:12b-it-qat'

In [28]:
client = OpenAI(base_url=api_key, api_key='')

In [29]:
adr = 'Тверская 10'

prompt = (
    "Проанализируй следующий нормализованный адрес. "
    "Дай дополнительную информацию, возможные интерпретации или уточнения.\n\n"
    f"Адрес: {adr}"
)

completion = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}]
)

APIConnectionError: Connection error.

In [30]:
from openai import OpenAI

client = OpenAI(
    base_url="http://127.0.0.1:11434/v1",
    api_key="ollama"   # можно любое слово, оно не проверяется
)

model_name = "gemma3:12b-it-qat"

adr = "Тверская 10"

prompt = (
    "Проанализируй следующий нормализованный адрес. "
    "Дай дополнительную информацию, возможные интерпретации или уточнения.\n\n"
    f"Адрес: {adr}"
)

completion = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
)

print(completion.choices[0].message["content"])


TypeError: 'ChatCompletionMessage' object is not subscriptable

In [43]:
completion.choices[0].message.content

'Нормализованный адрес "Тверская 10" достаточно прост, но его можно проанализировать и дополнить, чтобы получить более полную картину. Вот анализ и возможная дополнительная информация:\n\n**1. Основная информация:**\n\n*   **Тверская:**  Это название улицы. Тверская улица - одна из главных улиц Москвы, исторический центр города.\n*   **10:** Это номер дома.\n\n**2.  Дополнительная информация и возможные уточнения:**\n\n*   **Город:**  Без явно указанного города предполагается Москва, поскольку Тверская улица известна в Москве. Однако, в России есть другие населенные пункты с улицами "Тверская", поэтому лучше уточнить город.\n*   **Регион:** Москва, Москва.\n*   **Почтовый индекс:**  Почтовый индекс для Тверской улицы в Москве зависит от конкретного участка. Для дома 10, скорее всего, это один из следующих индексов:\n    *   10125:  (часть Тверской от ул. Охотный ряд до Большой Дмитровкой)\n    *   101991: (Ближе к площади Революции)\n    *  Лучше проверить почтовый индекс конкретно для

In [45]:
pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.1 MB/s  0:00:00

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas
df = pandas.read_csv('df_cleaned_with_names.csv')
df

,id,latitude,longitude,housenumber,street,name
0,389567667,55.918906,37.818866,27/21,улица карла маркса,"{'addr:housenumber': '27/21', 'addr:street': '..."
1,389567701,55.920442,37.819326,23/16,улица карла маркса,"{'addr:housenumber': '23/16', 'addr:street': '..."
2,389582528,55.920523,37.818495,16,улица карла маркса,"{'addr:housenumber': '16', 'addr:street': 'ули..."
3,623415761,55.741123,37.710229,12А с11,подъёмная улица,"{'addr:housenumber': '12А с11', 'addr:street':..."
4,729316625,55.874163,37.636353,9А,полярная улица,"{'addr:housenumber': '9А', 'addr:housenumber2'..."
...,...,...,...,...,...,...
253675,19824311,55.751038,37.450313,7,улица нижние мнёвники,"{'addr:housenumber': '7', 'addr:street': 'улиц..."
253676,19824844,55.749064,37.458265,20,улица нижние мнёвники,"{'addr:housenumber': '20', 'addr:street': 'ули..."
253677,19824855,55.743981,37.461061,12А,улица нижние мнёвники,"{'addr:housenumber': '12А', 'addr:postcode': '..."
253678,19824856,55.761190,37.471396,70,улица нижние мнёвники,"{'addr:housenumber': '70', 'addr:street': 'ули..."


In [11]:
list(df[df['id'] == 389567667]['name'])[0]

"{'addr:housenumber': '27/21', 'addr:street': 'улица Карла Маркса', 'building': 'yes'}"

In [2]:
import pandas
df = pandas.read_csv('df_main_with_names.csv')
df

,id,latitude,longitude,housenumber,street,name
0,389567667,55.918906,37.818866,27/21,улица Карла Маркса,"{'addr:housenumber': '27/21', 'addr:street': '..."
1,389567701,55.920442,37.819326,23/16,улица Карла Маркса,"{'addr:housenumber': '23/16', 'addr:street': '..."
2,389582528,55.920523,37.818495,16,улица Карла Маркса,"{'addr:housenumber': '16', 'addr:street': 'ули..."
3,623415761,55.741123,37.710229,12А с11,Подъёмная улица,"{'addr:housenumber': '12А с11', 'addr:street':..."
4,729316625,55.874163,37.636353,9А,Полярная улица,"{'addr:housenumber': '9А', 'addr:housenumber2'..."
...,...,...,...,...,...,...
253675,19824311,55.751038,37.450313,7,улица Нижние Мнёвники,"{'addr:housenumber': '7', 'addr:street': 'улиц..."
253676,19824844,55.749064,37.458265,20,улица Нижние Мнёвники,"{'addr:housenumber': '20', 'addr:street': 'ули..."
253677,19824855,55.743981,37.461061,12А,улица Нижние Мнёвники,"{'addr:housenumber': '12А', 'addr:postcode': '..."
253678,19824856,55.761190,37.471396,70,улица Нижние Мнёвники,"{'addr:housenumber': '70', 'addr:street': 'ули..."


In [4]:
import pandas as pd

In [14]:
datamain = pd.read_pickle('address_data_ngram.pkl')
datamain

,id,latitude,longitude,housenumber,street,address,normalized
0,389567667,55.918906,37.818866,27/21,улица карла маркса,улица карла маркса 27/21,улица карла маркса 27/21
1,389567701,55.920442,37.819326,23/16,улица карла маркса,улица карла маркса 23/16,улица карла маркса 23/16
2,389582528,55.920523,37.818495,16,улица карла маркса,улица карла маркса 16,улица карла маркса 16
3,623415761,55.741123,37.710229,12А с11,подъёмная улица,подъёмная улица 12А с11,подъемная улица 12а с11
4,729316625,55.874163,37.636353,9А,полярная улица,полярная улица 9А,полярная улица 9а
...,...,...,...,...,...,...,...
253675,19824311,55.751038,37.450313,7,улица нижние мнёвники,улица нижние мнёвники 7,улица нижние мневники 7
253676,19824844,55.749064,37.458265,20,улица нижние мнёвники,улица нижние мнёвники 20,улица нижние мневники 20
253677,19824855,55.743981,37.461061,12А,улица нижние мнёвники,улица нижние мнёвники 12А,улица нижние мневники 12а
253678,19824856,55.761190,37.471396,70,улица нижние мнёвники,улица нижние мнёвники 70,улица нижние мневники 70


In [19]:
row = datamain.loc[253675]

In [22]:
row['id']

np.int64(19824311)